In [2]:
# Set up all packages that may be used

import pandas as pd
import sys
import numpy as np
import scipy as sp
import pickle
import os

import matplotlib.pyplot as plt
from math import radians, cos, sin, asin, sqrt
import datetime
from sklearn.linear_model import LinearRegression
import seaborn as sns
sns.set(style="ticks")
%matplotlib inline

pd.set_option('display.max_columns', None)

# Establish crosswalk multipliers

A large assumption is being made that the GEOIDs remain consistent across the years (where applicable). That is, GEOID of a block group 100030001001 refers to the same block group (roughly speaking) geographically for GEOID 100030001001 in year 2020.

## 1990 >> 2020
### Transforming 1990 to 2020 data using 1990-2010 and 2010-2020 crosswalks

In [2]:
folder = r"C:\\Users\\jenki\\Documents\\School\\Thesis\\Data\\NHGIS Data\\Crosswalks\\GEOID\\"

#### Read in cleaned 1990-2010 crosswalk file and establish BG geoids

In [3]:
crosswalks90 = pd.read_csv(folder+"1990-2010\\cleanedcrosswalks_1990-2010.csv")

,GEOID90,GEOID10,WEIGHT,PAREA_VIA_BLK00
0,10003000100101,100030028001000,0.255638,0.255638
1,10003000100101,100030028001001,0.744362,0.744362


In [4]:
crosswalks90.drop('PAREA_VIA_BLK00', axis=1, inplace=True)
crosswalks90 = crosswalks90.astype({'GEOID90': str, 'GEOID10': str})

In [5]:
crosswalks90['GEOID90BG'] = crosswalks90['GEOID90'].str[:12]
crosswalks90['GEOID10BG'] = crosswalks90['GEOID10'].str[:12]

In [6]:
crosswalks90.head(2)

,GEOID90,GEOID10,WEIGHT,GEOID90BG,GEOID10BG
0,10003000100101,100030028001000,0.255638,100030001001,100030028001
1,10003000100101,100030028001001,0.744362,100030001001,100030028001


Check to confirm number of BG geoids for 1990 matches the one in 1990 database

In [7]:
checking90 = pd.read_excel("C:\\Users\\jenki\\Documents\\School\\Thesis\\Data\\NHGIS Data\\1990\\nhgis_1990_clean.xlsx", sheet_name="compiled", usecols=["geoid"])

In [8]:
checking90['geoidBG'] = checking90['geoid'].str[7:] 

In [9]:
checking90.head(2)

,geoid,geoidBG
0,15000US100030001001,100030001001
1,15000US100030001002,100030001002


In [10]:
len(checking90.geoidBG.unique()) == len(crosswalks90.GEOID90BG.unique())

True

#### Read in cleaned 2010-2020 crosswalk file and establish BG geoids

In [11]:
crosswalks10 = pd.read_csv(folder+"2010-2020\\cleanedcrosswalks_2010-2020.csv")
# crosswalks10.head(2)

In [12]:
crosswalks10.drop('PAREA', axis=1, inplace=True)
crosswalks10 = crosswalks10.astype({'GEOID10': str, 'GEOID20': str})
crosswalks10['GEOID10BG'] = crosswalks10['GEOID10'].str[:12]
crosswalks10['GEOID20BG'] = crosswalks10['GEOID20'].str[:12]
crosswalks10.head()

,GEOID10,GEOID20,WEIGHT,GEOID10BG,GEOID20BG
0,100030002001000,100030002001000,1.0,100030002001,100030002001
1,100030002001001,100030002001001,1.0,100030002001,100030002001
2,100030002001002,100030002001002,1.0,100030002001,100030002001
3,100030002001003,100030002001003,1.0,100030002001,100030002001
4,100030002001004,100030002001003,1.0,100030002001,100030002001


Check to confirm number of BG geoids matches the one in 2010 database

In [13]:
checking10 = pd.read_excel("C:\\Users\\jenki\\Documents\\School\\Thesis\\Data\\NHGIS Data\\2010\\nhgis_2010_clean.xlsx", sheet_name="compiled", usecols=["geoid"])

In [14]:
checking10['geoidBG'] = checking10['geoid'].str[7:] 

In [15]:
len(checking10.geoidBG.unique()) == len(crosswalks10.GEOID10BG.unique())

True

#### 1990-2010 to 2010-2020 join
Join 1990-2010 data to 2010-2020 data based on GEO10 Blocks (GEOID10). Fill NaN values with 1 and multiply the two crosswalk multipliers together to obtain multiplier value.

In [16]:
transform90 = crosswalks90.merge(crosswalks10, left_on='GEOID10', right_on='GEOID10', how='left')
transform90['WEIGHT_y'].fillna(1,inplace=True)
transform90['WEIGHT'] = transform90['WEIGHT_x']*transform90['WEIGHT_y']

In [17]:
# print(transform90.groupby('GEOID90BG').ngroups)
# returns 17380

In [18]:
weights = transform90.groupby(['GEOID90BG','GEOID20BG']).agg({'WEIGHT':'max'}, as_index=False)
print(weights)

                           WEIGHT
GEOID90BG    GEOID20BG           
100030001001 100030028001     1.0
100030001002 100030028001     1.0
             100030028002     1.0
100030001003 100030028002     1.0
100030001004 100030028001     1.0
...                           ...
540439558003 540439555004     1.0
             540439558003     1.0
             540439558004     0.0
540439558004 540439558003     1.0
             540439558004     1.0

[33873 rows x 1 columns]


In [19]:
weights.reset_index(inplace=True)

In [20]:
# Check for any null or na values in the multipliers dataframe. 
weights.isnull().any()

# Returns
# GEOID90BG    False
# GEOID20BG    False
# WEIGHT       False
# dtype: bool

GEOID90BG    False
GEOID20BG    False
WEIGHT       False
dtype: bool

In [21]:
# # If there were any true, we would count number of null/na values in the multipliers dataframe
# weights.isna().sum().sum()
# # Remove rows with na
# weights.dropna(inplace=True)

In [22]:
weights.to_csv(folder + "CrosswalkMultipliers\\multiplierweights90-20.csv",index=False)

## Check to see how the crosswalks transformation works using 2010 census data.

In [23]:
# folder = r"C:\\Users\\jenki\\Documents\\School\\Thesis\\Data\NHGIS Data\\"
# indicators = ['population', 'totalcivilianlaborforce','unemployedpopulation','totalhousingunits','vacanthousingunit']
# attributes = ['year','geoid','tract','state','county','msa','medianhouseholdincome','percapitaincome','mediancontractrent','mediangrossrent','medianhomevalue']
# usecols = indicators+attributes

In [24]:
# DEdata10 = pd.read_csv(folder+"Book2.csv", usecols=usecols)
# DEdata10 = DEdata10.fillna(0)
# DEdata10['geoid'] = DEdata10['geoid'].str[7:]
# len(DEdata10)

In [25]:
# crosswalks = pd.read_csv(folder+"nhgis_bg2010_bg2020_10\\nhgis_bg2010_bg2020_10.csv", usecols={'bg2010ge','bg2020ge','wt_pop'})
# crosswalks = crosswalks.fillna(0)
# crosswalks = crosswalks.astype({'bg2010ge':str, 'bg2020ge':str})
# crosswalks.rename(columns={'wt_pop':'weight'},inplace=True)
# len(crosswalks)

In [26]:
# merge = DEdata10.merge(crosswalks, left_on='geoid', right_on='bg2010ge', how='inner')

In [27]:
# for x in indicators:
#     merge[x]= merge[x]*merge['weight']

In [28]:
# merge = merge[merge['population'] >= 1]year

In [29]:
# merge['year'] = merge['year'].astype('datetime64[ns]')
# merge['tract'] = merge['tract'].astype('object')
# merge.dtypes

In [30]:
# group = merge.groupby(['bg2010ge']).agg({'bg2020ge':'max', 'year':'max','msa':'max','state':'max','county':'max','tract':'max',
#                                          'population':'sum', 'totalcivilianlaborforce':'sum', 'unemployedpopulation':'sum',
#                                          'totalhousingunits':'sum', 'vacanthousingunit':'sum',
#                                          'medianhouseholdincome':'max', 'percapitaincome':'max','mediancontractrent':'max', 'mediangrossrent':'max', 'medianhomevalue':'max',
#                                          })

In [31]:
# group.reset_index(inplace=True)

## 2000 >> 2020 multipliers
### Creating multipliers to transform 2000 to 2020 data using 2000-2010 and 2010-2020 crosswalks
Similar process to be followed as done for transforming and tracing 1990 to 2020 data. 

Reading in cleaned 2000-2010 crosswalk file and establish BG geoids

In [32]:
crosswalks00 = pd.read_csv(folder+"2000-2010\\cleanedcrosswalks_2000-2010.csv")
crosswalks00.head(2)

,GEOID00,GEOID10,WEIGHT,PAREA
0,100030001001000,100030028001000,1.0,1.0
1,100030001001001,100030028001001,1.0,1.0


In [33]:
crosswalks00.drop('PAREA', axis=1, inplace=True)
crosswalks00 = crosswalks00.astype({'GEOID00': str, 'GEOID10': str})
crosswalks00['GEOID00BG'] = crosswalks00['GEOID00'].str[:12]
crosswalks00['GEOID10BG'] = crosswalks00['GEOID10'].str[:12]
crosswalks00.head(2)

,GEOID00,GEOID10,WEIGHT,GEOID00BG,GEOID10BG
0,100030001001000,100030028001000,1.0,100030001001,100030028001
1,100030001001001,100030028001001,1.0,100030001001,100030028001


Check to confirm number of BG geoids matches the one in 2000 database

In [34]:
checking00 = pd.read_excel("C:\\Users\\jenki\\Documents\\School\\Thesis\\Data\\NHGIS Data\\2000\\nhgis_2000_clean.xlsx", sheet_name="compiled", usecols=["geoid"])

In [35]:
checking00['geoidBG'] = checking00['geoid'].str[7:] 
len(checking00.geoidBG.unique()) == len(crosswalks00.GEOID00BG.unique())

True

#### 2000-2010 to 2010-2020 data join
Join 2000-2010 data to 2010-2020 data based on GEO10 Blocks (GEOID10). Fill NaN values with 1 and multiply the two crosswalk multipliers together to obtain multiplier value.

In [36]:
transform00 = crosswalks00.merge(crosswalks10, left_on='GEOID10', right_on='GEOID10', how='left')

In [37]:
transform00['WEIGHT_y'].fillna(1,inplace=True)
transform00['WEIGHT'] = transform90['WEIGHT_x']*transform90['WEIGHT_y']

#### Extract multipliers (weights) for GEOID00 standardized to 2020
Group GEOID00BG by max weight (!!assumption. since multiple blocks per group...)

In [38]:
weights00 = transform00.groupby(['GEOID00BG','GEOID20BG']).agg({'WEIGHT': 'max'})
weights00.reset_index(inplace=True)
len(weights00)

29593

In [39]:
# Check for any null or na values in the multipliers dataframe. 
weights00.isnull().any()

# Returns
# GEOID90BG    False
# GEOID20BG    False
# WEIGHT       False
# dtype: bool

# This indicates that there are no values of GEOID00BG for which there is no corresponding GEOID20BG that will be the standardized equivalent.

GEOID00BG    False
GEOID20BG    False
WEIGHT       False
dtype: bool

In [40]:
weights00.to_csv(folder + "CrosswalkMultipliers\\multiplierweights00-20.csv",index=False)

### Extract multipliers for 2010-2020 data

In [41]:
crosswalks10

,GEOID10,GEOID20,WEIGHT,GEOID10BG,GEOID20BG
0,100030002001000,100030002001000,1.0,100030002001,100030002001
1,100030002001001,100030002001001,1.0,100030002001,100030002001
2,100030002001002,100030002001002,1.0,100030002001,100030002001
3,100030002001003,100030002001003,1.0,100030002001,100030002001
4,100030002001004,100030002001003,1.0,100030002001,100030002001
...,...,...,...,...,...
438327,540439558004188,540439558004005,1.0,540439558004,540439558004
438328,540439558004189,540439558004005,1.0,540439558004,540439558004
438329,540439558004190,540439558004075,1.0,540439558004,540439558004
438330,540439558004191,540439558004001,1.0,540439558004,540439558004


In [42]:
weights10 = crosswalks10.groupby(['GEOID10BG','GEOID20BG']).agg({'WEIGHT': 'max'})
weights10.reset_index(inplace=True)

In [43]:
# Check for any null or na values in the multipliers dataframe. 
weights10.isnull().any()

# Returns
# GEOID90BG    False
# GEOID20BG    False
# WEIGHT       False
# dtype: bool

# This indicates that there are no values of GEOID10BG for which there is no corresponding GEOID20BG that will be the standardized equivalent.

GEOID10BG    False
GEOID20BG    False
WEIGHT       False
dtype: bool

In [44]:
# Save weights file to csv for easy recall
weights10.to_csv(folder + "CrosswalkMultipliers\\multiplierweights10-20.csv",index=False)

# Data transformation

## Read in database pickles

In [3]:
# Read in database pickles and check
pickle_in = open('alldatadict.pickle', 'rb')
workingdict = pickle.load(pickle_in)
workingdict.keys()

dict_keys([2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 1990, 2000, 2010, 2011, 2012])

In [4]:
# Read in multiplier files
newfolder = r"C:\\Users\\jenki\\Documents\\School\\Thesis\\Data\NHGIS Data\\Crosswalks\\GEOID\\CrosswalkMultipliers\\"
x90 = pd.read_csv(newfolder + "multiplierweights90-20.csv")
x00 = pd.read_csv(newfolder + "multiplierweights00-20.csv")
x10 = pd.read_csv(newfolder + "multiplierweights10-20.csv")

In [5]:
# Establish variables to be used 
indicators = ['population', 'totalcivilianlaborforce','unemployedpopulation','totalhousingunits','vacanthousingunits']
attributes = ['year','tract','state','county','msa','medianhouseholdincome','percapitaincome','mediancontractrent','mediangrossrent','medianhomevalue']
usecols = indicators+attributes

## Filter and transform 1990 data

In [11]:
# Load 1990 data and check for na values
data90 = workingdict[1990]
data90.isna().sum().sum()

# If there are null values use the following:
# data90 = data90.fillna(0)b

0

In [12]:
# add new column 'geoidBG' for reference when performing merge and multiplication. 
data90['geoidBG'] = data90['geoid'].str[7:]

In [13]:
# Convert GEOID90BG and GEOID20BG to object type
x90 = x90.astype({'GEOID90BG':str, 'GEOID20BG':str}) 

# Convert percapitaincome into integer
data90['percapitaincome'] = data90['percapitaincome'].astype(int)

In [14]:
# Merge multiplier file to 1990 dataframe
merge90 = data90.merge(x90, left_on='geoidBG', right_on='GEOID90BG', how='inner')

In [15]:
# # Drop row if weight = 0 >> means that 1990 BG no longer exists in 2020 BG
# merge90 = merge90.loc[merge90['WEIGHT'] != 0]

In [16]:
# For all count-based indicators, multiply by weight from crosswalk
for x in indicators:
    merge90[x] = merge90[x]*merge90['WEIGHT']
    merge90[x].apply(np.ceil)
    merge90 = merge90.astype({x:int})

In [18]:
group90 = merge90.groupby(['GEOID20BG']).agg({'year':'max','msa':'max','state':'max','county':'max','tract':'max', 'blockgroup':'max',
                                         'population':'max', 'totalcivilianlaborforce':'max', 'unemployedpopulation':'max',
                                         'totalhousingunits':'max', 'vacanthousingunits':'max',
                                         'medianhouseholdincome':'max', 'percapitaincome':'max','mediancontractrent':'max', 'mediangrossrent':'max', 'medianhomevalue':'max', 'mediangrossretnaspercentageofhouseholdincome':'max'
                                         })

# Summation only makes sense if the crosswalks are given by block group, or if we are transforming the data from blocks.
# group90 = merge90.groupby(['GEOID20BG']).agg({'year':'max','msa':'max','state':'max','county':'max','tract':'max', 'blockgroup':'max',
#                                          'population':'sum', 'totalcivilianlaborforce':'sum', 'unemployedpopulation':'sum',
#                                          'totalhousingunits':'sum', 'vacanthousingunits':'sum',
#                                          'medianhouseholdincome':'max', 'percapitaincome':'max','mediancontractrent':'max', 'mediangrossrent':'max', 'medianhomevalue':'max',
#                                          })

In [19]:
group90.reset_index(inplace=True)

In [37]:
len(group90)

15992

In [26]:
# (group90['population'] == 0).sum()

## Filter and transform 2000 data

In [38]:
# Load 2000 data and check for na values
data00 = workingdict[2000]
data00.isna().sum().sum()

0

In [40]:
# add new column 'geoidBG' for reference when performing merge and multiplication. 
data00['geoidBG'] = data00['geoid'].str[7:]

In [41]:
# Convert GEOID00BG and GEOID20BG to object type
x00 = x00.astype({'GEOID00BG':str, 'GEOID20BG':str}) 

# Convert percapitaincome into integer
data00['percapitaincome'] = data00['percapitaincome'].astype(int)

In [42]:
# Merge multiplier file to 2000 dataframe
merge00 = data00.merge(x00, left_on='geoidBG', right_on='GEOID00BG', how='inner')

In [33]:
# Drop row if weight = 0 >> means that 2000 BG no longer exists in 2020 BG
# merge00 = merge00.loc[merge00['WEIGHT'] != 0]

In [44]:
# For all count-based indicators, multiply by weight from crosswalk
for x in indicators:
    merge00[x] = merge00[x]*merge00['WEIGHT']
    merge00[x].apply(np.ceil)
    merge00 = merge00.astype({x:int})

In [45]:
group00 = merge00.groupby(['GEOID20BG']).agg({'year':'max','msa':'max','state':'max','county':'max','tract':'max', 'blockgroup':'max',
                                         'population':'max', 'totalcivilianlaborforce':'max', 'unemployedpopulation':'max',
                                         'totalhousingunits':'max', 'vacanthousingunits':'max',
                                         'medianhouseholdincome':'max', 'percapitaincome':'max','mediancontractrent':'max', 'mediangrossrent':'max', 'medianhomevalue':'max', 'mediangrossretnaspercentageofhouseholdincome':'max'
                                         })
group00.reset_index(inplace=True)

In [47]:
len(group00)

11554

## Filter and transform 2010-2019 data

In [48]:
years = [2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019]

In [49]:
for year in years:
    print(workingdict[year].isna().sum().sum())

0
0
0
7
7
8017
8
6
9
8


In [50]:
x10 = x10.astype({'GEOID10BG':str, 'GEOID20BG':str}) 

In [52]:
for year in years:
    workingdict[year].fillna(0,inplace=True)
    workingdict[year]['geoidBG'] = workingdict[year]['geoid'].str[7:]
    workingdict[year]['percapitaincome'] = workingdict[year]['percapitaincome'].astype(int)

In [53]:
merge = {}
for year in years:
    merge[year] = workingdict[year].merge(x10, left_on='geoidBG', right_on='GEOID10BG', how='inner')
#     merge[year] = merge[year].loc[merge[year]['WEIGHT'] != 0]

In [54]:
for year in years:
    for x in indicators:
        merge[year][x] = merge[year][x]*merge[year]['WEIGHT']
        merge[year][x].apply(np.ceil)
        merge[year] = merge[year].astype({x:int})

In [55]:
dictgroup = {}

for year in years:
    group = merge[year].groupby(['GEOID20BG']).agg({'year':'max','msa':'max','state':'max','county':'max','tract':'max', 'blockgroup':'max',
                                         'population':'max', 'totalcivilianlaborforce':'max', 'unemployedpopulation':'max',
                                         'totalhousingunits':'max', 'vacanthousingunits':'max',
                                         'medianhouseholdincome':'max', 'percapitaincome':'max','mediancontractrent':'max', 'mediangrossrent':'max', 'medianhomevalue':'max', 'mediangrossretnaspercentageofhouseholdincome':'max'
                                         })
   
    group.reset_index(inplace=True)
    
    dictgroup[year] = group

In [56]:
# Copy dataframes for 1990 and 2000 data (grouped) into the new dictionary of data
dictgroup[1990] = group90.copy()
dictgroup[2000] = group00.copy()

#### Clean 2020 data into the same format (columns and all) to insert into the new dictionary of data

In [58]:
# Create new dataframe for 2020 data
data20 = workingdict[2020].copy()

In [59]:
# Add geoid column
data20['GEOID20BG'] = data20['geoid'].str[7:]

In [60]:
# Drop unneeded columns
data20.drop(['geoid', 'statefp', 'countyfp', 'statecountyfp'], axis = 1, inplace=True)

In [61]:
# Add to dictgroup dictionary
dictgroup[2020] = data20.copy()

#### Convert all dataframe datatypes into the same types

In [78]:
dtypes_new = group90.dtypes.to_dict()

In [79]:
keys = list(dictgroup.keys())
keys

[2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 1990, 2000, 2020]

In [86]:
for key in keys:
    dictgroup[key] = dictgroup[key].fillna(0).astype(dtypes_new)

#### Convert all negative values to 0

In [87]:
for year in keys:
    dictgroup[year].loc[dictgroup[year]["medianhouseholdincome"] < 0, "medianhouseholdincome"] = 0
    dictgroup[year].loc[dictgroup[year]["percapitaincome"] < 0, "percapitaincome"] = 0
    dictgroup[year].loc[dictgroup[year]["mediancontractrent"] < 0, "mediancontractrent"] = 0
    dictgroup[year].loc[dictgroup[year]["mediangrossrent"] < 0, "mediangrossrent"] = 0
    dictgroup[year].loc[dictgroup[year]["medianhomevalue"] < 0, "medianhomevalue"] = 0
    dictgroup[year].loc[dictgroup[year]["mediangrossretnaspercentageofhouseholdincome"] < 0, "mediangrossretnaspercentageofhouseholdincome"] = 0

In [88]:
dictgroup[2020].min()

year                                                      2020
population                                                   0
medianhouseholdincome                                        0
percapitaincome                                              0
totalcivilianlaborforce                                      0
unemployedpopulation                                         0
totalhousingunits                                            0
vacanthousingunits                                           0
mediancontractrent                                           0
mediangrossrent                                              0
mediangrossretnaspercentageofhouseholdincome               0.0
medianhomevalue                                              0
tract                                                      100
blockgroup                                                   0
msa                                                 Charleston
state                                                 D

#### Concatenate data from dictionary into dataframe

In [93]:
alldata = pd.concat(dictgroup.values())

Checking to see how many BG index values stay consistent over the years of data available

In [95]:
g = alldata.groupby(['GEOID20BG', 'year']).agg({'tract': 'count'}).reset_index()

In [97]:
g.groupby('year').agg({'GEOID20BG': 'count'})

,GEOID20BG
year,
1990,15992
2000,11554
2010,16002
2011,16002
2012,16002
2013,16002
2014,16002
2015,16002
2016,16002


In [98]:
g = g.groupby(['GEOID20BG']).agg({'year': 'count'}).reset_index()

In [99]:
g.groupby('year').agg({'GEOID20BG': 'count'})

,GEOID20BG
year,
11,30
12,4436
13,11536


# Save all the progress to pickles

In [102]:
# save dataframe "alldata" into pickle file
pickle_out = open('allcrosswalkeddatadf.pickle', 'wb')
pickle.dump(alldata, pickle_out)
pickle_out.close()

In [103]:
# print file size
print('File size of pickle file is', round(os.path.getsize('allcrosswalkeddatadf.pickle') / (1024**2), 1), 'MB')

File size of pickle file is 23.2 MB


In [ ]:
# save dictionary "dictgroup" into new pickle file
pickle_out = open('allcrosswalkeddictionary.pickle', 'wb')
pickle.dump(dictgroup, pickle_out)
pickle_out.close()

In [ ]:
# print file size
print('File size of pickle file is', round(os.path.getsize('crosswalkeddictionary.pickle') / (1024**2), 1), 'MB')